In [3]:
import pandas as pd
from openpyxl import Workbook
import numpy as np
# Read the data
all_data = pd.read_hdf('eodprices.h5')
beta_data = pd.read_excel('指数数据.xlsx')

# Convert dates to datetime objects
all_data['TRADE_DT'] = pd.to_datetime(all_data['TRADE_DT'], format='%Y%m%d')
all_data.set_index('TRADE_DT', inplace=True)
beta_data.set_index("日期", inplace=True)

# Reset index to ensure uniqueness
all_data.reset_index(drop=True, inplace=True)

# Resample data for weekly prices
all_data_weekly = all_data.groupby('S_INFO_WINDCODE')['S_DQ_ADJCLOSE'].resample('1W').last().ffill()


# Define a function to calculate metrics
def cal(stock_code, end_date):
    try:
        end_date = pd.Timestamp(end_date)
        end_date_idx = all_data.index.get_loc(end_date, method='ffill')

        start_date_1y_idx = end_date_idx - 250
        start_date_1m_idx = end_date_idx - 22
        start_date_3m_idx = end_date_idx - 66
        start_date_6m_idx = end_date_idx - 125

        if start_date_1y_idx < 0:
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

        stock_data = all_data_weekly.loc[stock_code].iloc[start_date_1y_idx:end_date_idx]
        stock_data_ret = stock_data.pct_change().dropna()

        volatility = stock_data_ret.std() * np.sqrt(52)
        mom_1m = (stock_data.iloc[-1] - stock_data.iloc[start_date_1m_idx]) / stock_data.iloc[start_date_1m_idx]
        mom_3m = (stock_data.iloc[-1] - stock_data.iloc[start_date_3m_idx]) / stock_data.iloc[start_date_3m_idx]
        mom_6m = (stock_data.iloc[-1] - stock_data.iloc[start_date_6m_idx]) / stock_data.iloc[start_date_6m_idx]

        beta_end_index = beta_data.index.get_loc(end_date, method='ffill')
        beta_data_1y = beta_data.iloc[beta_end_index - 250:beta_end_index]

        stock_data_ret = stock_data_ret.iloc[-len(beta_data_1y):]

        beta_hs300 = stock_data_ret.corr(beta_data_1y['沪深300'].pct_change().dropna())
        beta_zz500 = stock_data_ret.corr(beta_data_1y['中证500'].pct_change().dropna())

        return volatility, mom_1m, mom_3m, mom_6m, beta_hs300, beta_zz500
    except KeyError:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

# Read the stock list
all_stocks = pd.read_excel("stock_list.xlsx")
all_stock_lst = all_stocks["S_INFO_WINDCODE"].tolist()

# Create a Workbook and sheets
wb = Workbook()
ws1 = wb.active
ws1.title = '2018-06-30'
ws1.append(['S_INFO_WINDCODE', 'volatility', 'mom_1m', 'mom_3m', 'mom_6m', 'beta_hs300', 'beta_zz500'])
ws2 = wb.create_sheet(title='2018-12-31')
ws2.append(['S_INFO_WINDCODE', 'volatility', 'mom_1m', 'mom_3m', 'mom_6m', 'beta_hs300', 'beta_zz500'])

# Calculate and append data
for stock_code in all_stock_lst:
    data_0630 = cal(stock_code, '2018-06-30')
    data_1231 = cal(stock_code, '2018-12-31')
    ws1.append([stock_code] + list(data_0630))
    ws2.append([stock_code] + list(data_1231))

# Save the Workbook
wb.save("sample_res22.xlsx")


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'